## 지니차트 Top 200

![Alt text](image-6.png)

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import quote 

- Step 1

In [4]:
url = 'https://www.genie.co.kr/chart/top200'
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
res.text[:500]


'\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\n\n\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<!--[if lte IE 6]><html dir="ltr" class="no-js ie6"><![endif]-->\n<!--[if lte IE 7]><html dir="ltr" class="no-js ie7"><![endif]-->\n<!--[if IE 8]><html dir="ltr" class="no-js ie8"><![endif]-->\n<!--[if IE 9]><html dir="ltr" class="no-js ie9"><![endif]-->\n<!--[if (gte IE 10)|!(IE)]><!--><html dir="ltr" class="no-js"><!--<![endif]-->\n<title>음악, 그리고 설레임 - 지니</title>\n<meta charset="utf-8'

- 원리
    - html : BeautifulSoup (정적)
    - selenium (동적)  -> JS도 읽음

In [5]:
from selenium import webdriver
driver = webdriver.Chrome() 
driver.get(url)


In [6]:
# 크롬을 사용하여 불러오면 현재 지니에 적용된 시간(자바스크립트 데이터)을 불러올 수 있음
soup = BeautifulSoup(driver.page_source, 'html.parser')
date = soup.select_one('#inc_date').get_text().strip().replace('.', '')
time = soup.select_one('#inc_time').get_text().strip()[:2]
date, time

('20230919', '10')

![Alt text](image-8.png)

![Alt text](image-7.png)

In [7]:
# selenium 없으면 datetime.datetime.now()로.. 이용
import datetime
datetime.datetime.now()

datetime.datetime(2023, 9, 19, 10, 44, 37, 247412)

In [8]:
# Beautifulsoup 에서는 예전 시간만 불러옴 (HTML에 '#inc_date' 같은 id값이 있기 때문 )
date_str = soup.select_one('#inc_date').get_text().strip() + \
            soup.select_one('#inc_time').get_text().strip()
date_str = date_str.replace('.','').replace(':','')
date_str

'202309191000'

In [9]:
soup.select_one('.chart')

<div class="chart">
<div id="svgContainer"><span class="bg-line" style="left: 20px;"></span><span class="bg-line" style="left: 260px;"></span><svg height="290" style="overflow: hidden; position: relative; left: -0.5px; top: -0.5px;" version="1.1" width="502" xmlns="http://www.w3.org/2000/svg"><rect fill="url(#647F0B0E-5C52-41F8-8234-444C9C14A9E3)" height="290" r="0" rx="0" ry="0" stroke="none" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);" width="120" x="300" y="0"></rect><desc style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);">Created with Raphaël 2.1.0</desc><defs style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);"><pattern height="290" id="647F0B0E-5C52-41F8-8234-444C9C14A9E3" patterntransform="matrix(1,0,0,1,0,0) translate(300,0)" patternunits="userSpaceOnUse" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);" width="20" x="0" y="0"><image height="290" style="-webkit-tap-highlight-color: rgba(0, 0, 0, 0);" width="20" x="0" xlink:href="//image.genie.co.kr/imageg/w

- Step 2

In [10]:
# 하나의 데이터 
# trs = soup.select('tr.list')로 해도됨 
lis = soup.select('tbody > .list')
li = lis[10]
len(lis)

50

- Step 3

In [11]:
# 순위
rank = int(li.select_one('.number').get_text().replace(' ', '').split('\n')[0])
# rank = int(li.select_one('.number').get_text().strip().split('\n')[0]) 이것도됨 
rank

11

In [12]:
# 제목
title = li.select_one('.title.ellipsis').get_text().strip()
title

'사막에서 꽃을 피우듯'

In [13]:
# 뮤지션
artist = li.select_one('.artist.ellipsis').get_text().strip()
artist

'우디 (Woody)'

In [14]:
# 앨범
album = li.select_one('.albumtitle.ellipsis').get_text().strip()
album

'사막에서 꽃을 피우듯'

- Step 4

In [15]:
data = []
for li in lis:
    rank = int(li.select_one('.number').get_text().replace(' ', '').split('\n')[0])
    title = li.select_one('.title.ellipsis').get_text().strip()
    artist = li.select_one('.artist.ellipsis').get_text().strip()
    album = li.select_one('.albumtitle.ellipsis').get_text().strip()
    img = 'http:' + li.select_one('img')['src']
    data.append({'순위':rank, '제목':title, '가수':artist, '앨범':album, '이미지':img})

- Step 5 생략
- Step 6

In [16]:
data = []

for page in tqdm(range(1, 5)):
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh={time}&rtm=Y&pg={page}'
    res = requests.get(url, headers=header)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('tbody > .list')
    
    for li in lis:
        rank = int(li.select_one('.number').get_text().replace(' ', '').split('\n')[0])
        title = li.select_one('.title.ellipsis').get_text().strip()
        artist = li.select_one('.artist.ellipsis').get_text().strip()
        album = li.select_one('.albumtitle.ellipsis').get_text().strip()
        img = 'http:' + li.select_one('img')['src']
        data.append({'순위':rank, '제목':title, '가수':artist, '앨범':album, '이미지':img})


 25%|██▌       | 1/4 [00:00<00:01,  2.00it/s]

100%|██████████| 4/4 [00:01<00:00,  2.61it/s]


- Step 7

In [17]:
df = pd.DataFrame(data)
df

,순위,제목,가수,앨범,이미지
0,1,Love Lee,AKMU (악뮤),Love Lee,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
1,2,Smoke (Prod. by Dynamicduo & Padi),다이나믹 듀오 & 이영지,스트릿 우먼 파이터2 (SWF2) 계급미션,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
2,3,후라이의 꿈,AKMU (악뮤),Love Lee,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
3,4,ETA,NewJeans,NewJeans 2nd EP 'Get Up',http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
4,5,Seven (Feat. Latto) (Clean Ver.),정국,Seven (Clean Ver.),http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
...,...,...,...,...,...
195,196,I'm Not The Only One,Sam Smith,In The Lonely Hour (Standard Edition),http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/080...
196,197,혼자라고 생각말기,임한별,혼자라고 생각말기,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083...
197,198,Way Back Home,숀 (SHAUN),Take,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/081...
198,199,Memories,RIIZE,Get A Guitar,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...


In [18]:
driver.close()

In [19]:
print(date+time)

2023091910


In [20]:
filename = date+time + '.csv'
filename

'2023091910.csv'

In [21]:
df.to_csv('data/지니차트_'+filename, index=False)
pd.read_csv('data/지니차트_'+filename).tail()

,순위,제목,가수,앨범,이미지
195,196,I'm Not The Only One,Sam Smith,In The Lonely Hour (Standard Edition),http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/080...
196,197,혼자라고 생각말기,임한별,혼자라고 생각말기,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083...
197,198,Way Back Home,숀 (SHAUN),Take,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/081...
198,199,Memories,RIIZE,Get A Guitar,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
199,200,사랑이라 쓰고 이별이라 읽어,송하예,사랑이라 쓰고 이별이라 읽어,http://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084...
